In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/tanh'

In [4]:
# !pip install smt

In [5]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y

In [6]:
loss_thresh = 0.005
label = "1D_FODE_tanh_" 

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [7]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
             
      
              
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) 
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
   
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time()
    thresh_flag = 0

    x_coll_np_array = colloc_pts(N_f,123)
    x_coll = torch.from_numpy(x_coll_np_array).float().to(device)

    f_hat = torch.zeros(x_coll.shape[0],1).to(device)

    for i in range(max_iter):
        train_step(i)        
    
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
 
max_reps = 10
max_iter = 100

train_loss_full = []
test_mse_full = []
test_re_full = []

elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):   
  train_loss = []
  test_mse_loss = []
  test_re_loss = []   


  torch.manual_seed(reps*36)
  N_f = 10000 #Total number of collocation points

  layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
  PINN = Sequentialmodel(layers)

  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())

  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                            max_iter = 10, 
                            max_eval = 15, 
                            tolerance_grad = 1e-6, 
                            tolerance_change = 1e-6, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')



  train_model(max_iter,reps)


  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)


  print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.768238 Test MSE 5220.0045811306 Test RE 0.9997435404560636
1 Train Loss 47.76683 Test MSE 5215.485107216673 Test RE 0.9993106583394861
2 Train Loss 47.749943 Test MSE 5174.956820242627 Test RE 0.9954203839808021
3 Train Loss 47.748604 Test MSE 5177.407936872666 Test RE 0.995656096344

0 Train Loss 47.76833 Test MSE 5220.246779464616 Test RE 0.9997667332912018
1 Train Loss 47.76683 Test MSE 5215.403358712495 Test RE 0.9993028266162653
2 Train Loss 47.74634 Test MSE 5169.998287146971 Test RE 0.9949433744127183
3 Train Loss 47.745472 Test MSE 5172.130720942068 Test RE 0.9951485419995706
4 Train Loss 47.73545 Test MSE 5162.088103227977 Test RE 0.994181943042029
5 Train Loss 47.320904 Test MSE 4937.630695207722 Test RE 0.9723272713310455
6 Train Loss 45.09703 Test MSE 5032.301825908832 Test RE 0.981604420005369
7 Train Loss 39.49209 Test MSE 3591.2789506052623 Test RE 0.8292352107240546
8 Train Loss 37.12409 Test MSE 3384.402134727949 Test RE 0.8049967830551102
9 Train Loss 34.81187 Test MSE 2953.7969472592463 Test RE 0.752044402850827
10 Train Loss 34.250458 Test MSE 2713.4698363884495 Test RE 0.7208014701928259
11 Train Loss 30.888367 Test MSE 2432.6465318995424 Test RE 0.6824843067592082
12 Train Loss 25.50415 Test MSE 1805.3650509191475 Test RE 0.5879436174441085
13 

0 Train Loss 47.76986 Test MSE 5223.414372220588 Test RE 1.0000700114443049
1 Train Loss 47.749126 Test MSE 5172.878094242078 Test RE 0.9952204389240698
2 Train Loss 47.74749 Test MSE 5173.7957546073485 Test RE 0.9953087102727373
3 Train Loss 47.73213 Test MSE 5139.70777402239 Test RE 0.9920244548703359
4 Train Loss 47.716015 Test MSE 5102.915274790106 Test RE 0.9884673836417674
5 Train Loss 47.53399 Test MSE 4933.081081836458 Test RE 0.9718792089993209
6 Train Loss 47.374886 Test MSE 4816.201787099489 Test RE 0.960296844816548
7 Train Loss 47.25087 Test MSE 4739.830773552604 Test RE 0.9526526564880398
8 Train Loss 47.08384 Test MSE 4661.3479504694715 Test RE 0.9447326522802131
9 Train Loss 46.953 Test MSE 4605.147642990859 Test RE 0.9390202196192485
10 Train Loss 46.585182 Test MSE 4560.609716291024 Test RE 0.9344683976591216
11 Train Loss 44.498814 Test MSE 4478.251681154761 Test RE 0.9259923812799068
12 Train Loss 42.410885 Test MSE 4417.653240630643 Test RE 0.9197059080925345
13 Tr

0 Train Loss 47.768715 Test MSE 5221.043838829687 Test RE 0.9998430556429755
1 Train Loss 47.743134 Test MSE 5164.947947156842 Test RE 0.9944572978457233
2 Train Loss 47.74143 Test MSE 5163.230824017293 Test RE 0.9942919769552846
3 Train Loss 47.720394 Test MSE 5126.454117811679 Test RE 0.9907445729501702
4 Train Loss 47.36402 Test MSE 4996.093078703556 Test RE 0.9780665924550836
5 Train Loss 45.78136 Test MSE 4709.86349938025 Test RE 0.9496363388402131
6 Train Loss 44.16574 Test MSE 4562.935694415078 Test RE 0.9347066636060455
7 Train Loss 43.278908 Test MSE 4443.833509434002 Test RE 0.9224271017232776
8 Train Loss 41.14781 Test MSE 4647.806940145601 Test RE 0.9433594509270246
9 Train Loss 35.73835 Test MSE 4810.049962617007 Test RE 0.9596833462992305
10 Train Loss 33.18985 Test MSE 5604.415137707833 Test RE 1.0359011457118938
11 Train Loss 23.761715 Test MSE 3857.932682287492 Test RE 0.8594695353092996
12 Train Loss 14.251568 Test MSE 1675.505427599507 Test RE 0.566403703251464
13 Tr

0 Train Loss 47.770046 Test MSE 5223.777237876034 Test RE 1.0001047477998886
1 Train Loss 47.75167 Test MSE 5184.9343322450695 Test RE 0.9963795259169129
2 Train Loss 47.74747 Test MSE 5169.320956164997 Test RE 0.9948781975974441
3 Train Loss 47.720303 Test MSE 5107.116251177536 Test RE 0.9888741779567803
4 Train Loss 47.675224 Test MSE 5041.676962192721 Test RE 0.9825183549593651
5 Train Loss 47.543373 Test MSE 4774.745057925337 Test RE 0.9561549079626939
6 Train Loss 47.478046 Test MSE 4716.798159766254 Test RE 0.950335189586729
7 Train Loss 47.421055 Test MSE 4784.9206953069415 Test RE 0.9571732144172671
8 Train Loss 46.043327 Test MSE 4844.739039419056 Test RE 0.9631376475990987
9 Train Loss 44.891804 Test MSE 4452.242083434956 Test RE 0.9232993926069077
10 Train Loss 40.512493 Test MSE 4682.353928472449 Test RE 0.9468589391674554
11 Train Loss 35.58157 Test MSE 4399.0527433530115 Test RE 0.9177676581294515
12 Train Loss 34.7577 Test MSE 4282.309447173213 Test RE 0.9055077843767687

0 Train Loss 47.75011 Test MSE 5179.706530058572 Test RE 0.995877090555358
1 Train Loss 47.731953 Test MSE 5127.2764005706385 Test RE 0.9908240274323943
2 Train Loss 47.70203 Test MSE 5076.085420727557 Test RE 0.9858654016747049
3 Train Loss 47.57816 Test MSE 4879.338052336487 Test RE 0.9665706836684679
4 Train Loss 47.49621 Test MSE 4769.664598588262 Test RE 0.9556460850892353
5 Train Loss 47.12491 Test MSE 4758.507697402498 Test RE 0.9545277369305885
6 Train Loss 46.760616 Test MSE 4909.5187491123315 Test RE 0.9695553924079181
7 Train Loss 46.378582 Test MSE 5303.801386254313 Test RE 1.0077360406155358
8 Train Loss 44.463696 Test MSE 5068.146490214029 Test RE 0.9850941597740579
9 Train Loss 39.483383 Test MSE 4238.979809227507 Test RE 0.9009150431441624
10 Train Loss 32.64055 Test MSE 3309.9336824622346 Test RE 0.7960911773015125
11 Train Loss 31.083952 Test MSE 3026.8683233693296 Test RE 0.7612896560410621
12 Train Loss 30.02726 Test MSE 3076.8176312568517 Test RE 0.7675453455807418

0 Train Loss 47.771984 Test MSE 5227.58558683834 Test RE 1.000469240175276
1 Train Loss 47.771347 Test MSE 5226.127185446533 Test RE 1.0003296740709156
2 Train Loss 47.76545 Test MSE 5214.164662883707 Test RE 0.9991841487628828
3 Train Loss 47.756462 Test MSE 5190.605182057624 Test RE 0.9969242555285143
4 Train Loss 47.741104 Test MSE 5155.810820645249 Test RE 0.9935772788759486
5 Train Loss 47.71278 Test MSE 5081.400992166431 Test RE 0.9863814555169698
6 Train Loss 47.6619 Test MSE 5016.245645161774 Test RE 0.9800372038147908
7 Train Loss 47.2341 Test MSE 4802.566804995854 Test RE 0.9589365497910671
8 Train Loss 45.791756 Test MSE 4781.2672367997675 Test RE 0.9568077266045586
9 Train Loss 38.93052 Test MSE 3489.666396036463 Test RE 0.8174197381780395
10 Train Loss 36.10155 Test MSE 3155.4872480754625 Test RE 0.7772959044077303
11 Train Loss 33.2497 Test MSE 2589.51977967251 Test RE 0.7041461045003778
12 Train Loss 29.505274 Test MSE 1986.6335872715947 Test RE 0.6167541089063078
13 Tra

0 Train Loss 47.76869 Test MSE 5221.031392931627 Test RE 0.9998418639318398
1 Train Loss 47.75109 Test MSE 5181.3665632642505 Test RE 0.9960366610439519
2 Train Loss 47.70971 Test MSE 5078.335452534758 Test RE 0.986083875416609
3 Train Loss 47.18677 Test MSE 4822.618018010473 Test RE 0.9609362943066958
4 Train Loss 44.538017 Test MSE 4538.495665130863 Test RE 0.9322000613212892
5 Train Loss 44.161808 Test MSE 4502.4953993159515 Test RE 0.928495500622295
6 Train Loss 40.594315 Test MSE 3861.237711087328 Test RE 0.8598376033515814
7 Train Loss 34.202084 Test MSE 4467.473206075826 Test RE 0.9248773480800233
8 Train Loss 30.040089 Test MSE 4747.219161893507 Test RE 0.9533948588307345
9 Train Loss 28.254715 Test MSE 5112.279887506996 Test RE 0.9893739606479761
10 Train Loss 27.01249 Test MSE 5122.20619912383 Test RE 0.9903340089919782
11 Train Loss 26.623402 Test MSE 5133.765956602048 Test RE 0.9914508684903169
12 Train Loss 25.611406 Test MSE 5809.3420270039605 Test RE 1.054670109756035
13

0 Train Loss 47.767467 Test MSE 5218.289043795572 Test RE 0.9995792457432063
1 Train Loss 47.766663 Test MSE 5216.33249022413 Test RE 0.9993918362606806
2 Train Loss 47.747005 Test MSE 5158.200718315839 Test RE 0.99380753101274
3 Train Loss 47.74459 Test MSE 5165.172852814558 Test RE 0.9944789492389414
4 Train Loss 47.740337 Test MSE 5163.230642010044 Test RE 0.9942919594305646
5 Train Loss 47.681576 Test MSE 5047.218339712777 Test RE 0.9830581564931334
6 Train Loss 47.381878 Test MSE 4781.069252124668 Test RE 0.9567879164560587
7 Train Loss 46.850075 Test MSE 4766.767900612809 Test RE 0.9553558510018285
8 Train Loss 45.469128 Test MSE 4682.011958122483 Test RE 0.9468243621576856
9 Train Loss 44.557312 Test MSE 4510.282043583845 Test RE 0.9292980268198218
10 Train Loss 39.7893 Test MSE 4236.105512733144 Test RE 0.9006095526515957
11 Train Loss 33.873573 Test MSE 2845.718831575284 Test RE 0.7381577071267779
12 Train Loss 29.52197 Test MSE 2361.4629235268803 Test RE 0.6724248121385549
13

0 Train Loss 47.770065 Test MSE 5223.797661906716 Test RE 1.0001067029129378
1 Train Loss 47.748383 Test MSE 5177.246390263638 Test RE 0.9956405628846065
2 Train Loss 47.700153 Test MSE 5067.736903803895 Test RE 0.9850543533740684
3 Train Loss 47.497715 Test MSE 4858.461411834619 Test RE 0.9645006919129846
4 Train Loss 46.063244 Test MSE 4991.506745467653 Test RE 0.9776175646666522
5 Train Loss 37.97675 Test MSE 4543.129478206785 Test RE 0.9326758289681283
6 Train Loss 34.745377 Test MSE 4297.908440096958 Test RE 0.9071555132391553
7 Train Loss 32.70767 Test MSE 4457.632074008714 Test RE 0.9238581077048926
8 Train Loss 31.650553 Test MSE 4585.611131915883 Test RE 0.9370262904062169
9 Train Loss 30.22256 Test MSE 4893.29248752734 Test RE 0.9679518462387207
10 Train Loss 28.689322 Test MSE 5267.418083209872 Test RE 1.0042736314477345
11 Train Loss 28.43971 Test MSE 5405.078831682399 Test RE 1.017312031642028
12 Train Loss 27.473633 Test MSE 5509.997162846097 Test RE 1.027138131538264
13 

In [13]:
import scipy.io as sio
import numpy as np

In [14]:
for tune_reps in range(5):
    label = "1D_FODE_tanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: '1D_FODE_tanh_tune0.mat'